<a href="https://colab.research.google.com/github/pamelamurilloa/DataMining-ISW911-ThirdProject-MachineLearning/blob/main/ProyectoIII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto III - Machine Learning

## 0. Descripción de las variables del dataset

1. **churn:** Indica si un cliente ha abandonado el servicio o no.
2. **accountlength:** Tiempo, en días, que el cliente lleva con el servicio.
3. **internationalplan:** Indica si el cliente tiene un plan de llamadas internacionales.
4. **voicemailplan:** Indica si el cliente tiene un plan de buzón de voz.
5. **numbervmailmessages:** Número de mensajes de voz recibidos por el cliente.
6. **totaldayminutes:** Total de minutos usados por el cliente durante el día.
7. **totaldaycalls:** Total de llamadas realizadas durante el día.
8. **totaldaycharge:** Costo total de las llamadas realizadas durante el día, en dólares.
9. **totaleveminutes:** Total de minutos usados por el cliente durante la tarde.
10. **totalevecalls:** Total de llamadas realizadas durante la tarde.
11. **totalevecharge:** Costo total de las llamadas realizadas durante la tarde, en dólares.
12. **totalnightminutes:** Total de minutos usados por el cliente durante la noche.
13. **totalnightcalls:** Total de llamadas realizadas durante la noche.
14. **totalnightcharge:** Costo total de las llamadas realizadas durante la noche, en dólares.
15. **totalintlminutes:** Total de minutos usados en llamadas internacionales.
16. **totalintlcalls:** Total de llamadas internacionales realizadas.
17. **totalintlcharge:** Costo total de las llamadas internacionales, en dólares.
18. **numbercustomerservicecalls:** Número de llamadas realizadas por el cliente al servicio de atención al cliente.

## 0.1. Cargar librerías previamente

In [2]:
# Carga de las librerias
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error, classification_report, mean_absolute_error, r2_score
import matplotlib.pyplot as plt

## 1. Cargar el Conjunto de datos


### 1.1. Cargar datos desde el repositorio a un dataframe

In [3]:
url ='https://raw.githubusercontent.com/pamelamurilloa/DataMining-ISW911-ThirdProject-MachineLearning/refs/heads/main/datos.csv'
df = pd.read_csv(url)

### 1.2. Transformaciones previas

In [4]:
# Validar como vienen originalmente los tipos datos
print(df.dtypes)

churn                          object
accountlength                 float64
internationalplan              object
voicemailplan                  object
numbervmailmessages           float64
totaldayminutes               float64
totaldaycalls                 float64
totaldaycharge                float64
totaleveminutes               float64
totalevecalls                 float64
totalevecharge                float64
totalnightminutes             float64
totalnightcalls               float64
totalnightcharge              float64
totalintlminutes              float64
totalintlcalls                float64
totalintlcharge               float64
numbercustomerservicecalls    float64
dtype: object


In [5]:
# Convertir las variables categóricas a un formato category (equivalente a factor en R)
columns_to_convert = ['churn', 'internationalplan', 'voicemailplan']
df[columns_to_convert] = df[columns_to_convert].astype('category')
# En caso de importar el orden aplicar: df['churn'] = pd.Categorical(df['churn'], categories=['No', 'Yes'], ordered=True)

print(df.dtypes)

churn                         category
accountlength                  float64
internationalplan             category
voicemailplan                 category
numbervmailmessages            float64
totaldayminutes                float64
totaldaycalls                  float64
totaldaycharge                 float64
totaleveminutes                float64
totalevecalls                  float64
totalevecharge                 float64
totalnightminutes              float64
totalnightcalls                float64
totalnightcharge               float64
totalintlminutes               float64
totalintlcalls                 float64
totalintlcharge                float64
numbercustomerservicecalls     float64
dtype: object


## 2. División de los datos en conjuntos de entrenamiento y prueba

In [12]:
# Separar el conjunto de datos en un conjunto de entrenamiento y uno de prueba

# Esta separación esta enfocada en la predección de 'churn'

# Variables independientes:
x = df.loc[:, df.columns != 'churn']

# Variable dependiente:
y = df['churn']

# Dividir los datos en subconjuntos aleatorios de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)
print('Conjunto de datos separado para entrenamiento y prueba.')

Conjunto de datos separado para entrenamiento y prueba.


## 3. Selección del modelo de Machine Learning

### 3.1. Justificación de los modelos seleccionados:

### 3.2. Implementación de Machine Learning




## Redes Neuronales



## Random Forest

In [ ]:
## Bootstraping


X = df.drop('{columna}', axis=1)
y = df['{columna}']

# División de datos entre prueba y entrenamiento

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0) # El 0.3 puede cambiarse a 0.1 o 0.2, podemos ir probando

### Regresión

In [ ]:
forest_reg = RandomForestRegressor(
                                    n_estimators=25, # Se puede incrementar si se ve que lo ocupamos, normalmente se utiliza 100 pero esto hace las pruebas mas rapidas
                                    random_state=1,
                                    max_features='sqrt',
                                    max_depth=10
                                  )

# Entrenando el modelo
forest_reg.fit(X_train, y_train)

# Make predictions
y_pred = forest_reg.predict(X_test)

# Evaluando el modelo
error_cuadrado = mean_squared_error(y_test, y_pred)
error_absoluto = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", error_cuadrado)
print("Mean Absolute Error (MAE):", error_absoluto)
print("R-squared (R²):", r2)
print('Train R²: %.5f' % forest_reg.score(X_train, y_train))
print('Test R²: %.5f' % forest_reg.score(X_test, y_test))

### Clasificación



In [ ]:
forest_cla = RandomForestClassifier(
                                    n_estimators=25,
                                    random_state=1,
                                    criterion='entropy', # Se puede cambiar a gini para ver si da mejor resultado
                                    max_features='sqrt',
                                    max_depth=10
                                  )

# Entrenando el modelo
forest_cla.fit(X_train, y_train)

# Hacer predicciones
y_pred = forest_cla.predict(X_test)

## Evaluando el modelo
accuracy = accuracy_score(y_test, y_pred)

print('Train Accuracy : %.5f' % forest_cla.score(X_train, y_train))
print('Test Accuracy : %.5f' % forest_cla.score(X_test, y_test))
print(classification_report(y_test, y_pred))

## No se tienen que poner todas las formas de evaluacion, solo la que nos funcione mas

# Videos que vi para ayudar con esto (esta parte luego se quita)


*   https://www.youtube.com/watch?v=AYICIq5jnhU
*   https://www.youtube.com/watch?v=3pa0vtW64Ic

# Para las redes neuronales:
*   https://www.youtube.com/watch?v=CU24iC3grq8
*   https://www.youtube.com/watch?v=iX_on3VxZzk
*   https://www.youtube.com/watch?v=V_pfbVudr8w